In [4]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from mpl_toolkits.axes_grid1 import ImageGrid
from torchvision.utils import save_image, make_grid


# 各データの
transform = transforms.Compose([transforms.ToTensor()])

path = "~/datasets"
train_dataset = MNIST(path, transform=transform, download=True)
test_dataset = MNIST(path, transform=transform, download=True)

batch_size = 100
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class VAE(nn.Module):

    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=200, device=device):
        super(VAE, self).__init__()
        self.device = device
        self.encode = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, latent_dim),
            nn.LeakyReLU(0.2)
        )

        # mean, log_var
        self.mean_layer = nn.Linear(latent_dim, 2)
        self.logvar_layer = nn.Linear(latent_dim, 2)

        # decoder
        self.decoder = nn.Sequential(
            # layer1
            nn.Linear(2, latent_dim),
            nn.LeakyReLU(0.2),
            # layer2
            nn.Linear(latent_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            # layer3
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid(),
        )

    def encode(self, x):
        x = self.encoder(x)
        mean, logvar = self.mean_layer(x), self.logvar_layer(x)
        return mean, logvar
    
    def reparemeterization(self, mean, var):
        epsilon = torch.randn_like(var).to(self.device)
        z = mean + var * epsilon
        return z
    
    def decode(self, x):
        return self.decode

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparemeterization(mean, logvar)
        x_hat = self.decode(z)
        return x_hat, mean, logvar
    

In [7]:
from torch.optim import Adam


model = VAE().to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

def loss_function(x, x_hat, mean, logvar):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return reproduction_loss + KLD

In [16]:
_, x = next(iter(train_loader))

In [19]:
model

tensor([6, 9, 6, 4, 8, 3, 5, 4, 7, 8, 4, 9, 3, 7, 0, 0, 5, 6, 1, 3, 1, 9, 3, 0,
        7, 4, 8, 9, 8, 1, 6, 9, 0, 0, 6, 6, 1, 1, 3, 5, 8, 6, 5, 8, 4, 9, 3, 8,
        6, 8, 7, 3, 1, 1, 8, 0, 3, 0, 1, 0, 7, 6, 8, 2, 4, 7, 6, 6, 1, 3, 4, 4,
        9, 2, 1, 7, 7, 5, 4, 0, 8, 5, 8, 5, 3, 1, 5, 4, 5, 7, 5, 9, 7, 5, 1, 4,
        6, 0, 1, 8])

In [8]:
def train(model, optimizer, epochs, device):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0.
        for batch_idx, (x, _) in enumerate(train_loader):
            x = x.view(batch_size, x_dim).to(device)

            optimizer.zero_grad()

            x_hat, mean, log_var = model(x)

            loss = loss_function(x, x_hat, mean, log_var)

            overall_loss += loss.item()

            loss.backward()
            optimizer.step()

        print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss / (batch_idx * batch_size))